# E1. Análisis Exploratorio y Preparación del Dataset

**Objetivo:**  
Familiarizarse con una serie temporal real de consumo de recursos (energía o agua), analizar sus componentes y preparar el conjunto de datos para su uso en modelos predictivos, incluyendo la creación de los conjuntos de entrenamiento y validación.

**Duración estimada:** 1h
En este primer ejercicio trabajaremos con **series temporales reales** relacionadas con el consumo de energía. El objetivo es preparar el conjunto de datos para su uso posterior en modelos predictivos.

## Tecnologías utilizadas

- **Pandas**: Para carga, procesamiento y análisis de datos tabulares.
- **Matplotlib & Seaborn**: Para generar visualizaciones estáticas de las series y relaciones entre variables.
- **Statsmodels**: Para realizar la descomposición de la serie en sus componentes (tendencia, estacionalidad y residuo).
- **Python**: Todo el trabajo se realizará en Jupyter o Google Colab con scripts en Python.

## ¿Qué es una serie temporal?

Una **serie temporal** es una secuencia de datos medidos en intervalos regulares de tiempo (diarios, mensuales, anuales...). Su análisis nos permite descubrir patrones como:

- Tendencias a largo plazo
- Estacionalidades o repeticiones periódicas
- Cambios o anomalías

## Flujo de trabajo del ejercicio

```mermaid
flowchart TD
    A[Dataset bruto: consumo energético] --> B[Conversión a frecuencia diaria]
    B --> C[Análisis exploratorio]
    C --> D[Visualización y correlación entre variables]
    D --> E[Descomposición de la serie]
    E --> F[División en conjuntos de entrenamiento y validación]
    F --> G[Datos listos para modelado predictivo]
```
---

## Paso 1: Cargar el dataset

Utilizaremos un dataset con datos diarios de consumo de agua por vivienda o edificio.

In [ ]:
import pandas as pd

# Cargar dataset desde archivo CSV
df = pd.read_csv("data/m5_consumo_energia.csv", parse_dates=["dt"])

# Establecer columna temporal como índice
df.set_index("dt", inplace=True)

# Agregamos datos a diario
df = df.groupby(df.index.date).agg({
    'Energia Consumida':'sum',
    'Temperatura':'mean',
    'Humedad':'mean',
})
df = df.round(3)

# Guardar datos agregados
df.to_csv('data/m5_consumo_energia_diario.csv')

# Mostrar las primeras filas
df.head()

In [ ]:
df.info()

In [ ]:
# Estadísticas descriptivas
df.describe()

> Asegúrate de que el archivo contiene al menos dos columnas:  
> - `fecha`: columna de tipo fecha  
> - `consumo`: valor numérico del consumo diario (litros o m³)

---

## Paso 2: Visualización de los datos

In [ ]:
import matplotlib.pyplot as plt
# Visualizar las variables clave
df.plot(subplots=True, figsize=(10, 8), grid=True)
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calcular matriz de correlación
corr = df.corr()

# Representar heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matriz de correlación entre variables")
plt.tight_layout()
plt.show()


---

## Paso 3: Descomposición de la serie

Descomponemos la serie en **tendencia, estacionalidad y residuo** para analizar su comportamiento.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(df["Energia Consumida"], model='additive', period=7)  # Periodo semanal

result.plot()
plt.suptitle("Descomposición de la serie temporal", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(df["Temperatura"], model='additive', period=7)  # Periodo semanal

result.plot()
plt.suptitle("Descomposición de la serie temporal", fontsize=14)
plt.tight_layout()
plt.show()

---

## Paso 4: Generación de conjuntos de entrenamiento y validación

Vamos a dividir la serie temporal en dos partes:
- **Conjunto de entrenamiento**: ~80% inicial de los datos.
- **Conjunto de validación**: ~20% final, que usaremos para evaluar las predicciones.

In [ ]:
# Definir punto de corte
split_index = int(len(df) * 0.9)

train_df = df.iloc[:split_index]
val_df = df.iloc[split_index:]

train_df.to_csv('data/m5_consumo_energia_train.csv')
val_df.to_csv('data/m5_consumo_energia_test.csv')

# Verificar tamaños
print(f"Entrenamiento: {len(train_df)} muestras")
print(f"Validación: {len(val_df)} muestras")

Visualizamos ambas particiones:

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(train_df.index, train_df["Energia Consumida"], label="Train")
plt.plot(val_df.index, val_df["Energia Consumida"], label="Validation")
plt.title("División de la serie en entrenamiento y validación")
plt.xlabel("Fecha")
plt.ylabel("Consumo")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

---

## ✅ Resultado

- Una serie temporal descompuesta y visualizada para su análisis.
- Un conjunto de datos limpio, indexado por fecha.
- Las particiones `train` y `val` listas para usar en modelos de predicción.